In [11]:
import torch
import torchvision
from torchvision import transforms
from icecream import ic
from torch.utils.data import DataLoader

In [8]:
DATASET_PATH = "../data"
# Path to the folder where the pretrained models are saved
CHECKPOINT_PATH = "../saved_models/vit"

In [13]:


# Load CIFAR-10 training data
transform = transforms.Compose([transforms.ToTensor()])
train_set = torchvision.datasets.CIFAR10(root=DATASET_PATH, train=True, download=True, transform=transform)
train_loader = DataLoader(train_set, batch_size=len(train_set), shuffle=False)

# Calculate mean and standard deviation
data = next(iter(train_loader))[0]
# ic(data.shape)
mean = torch.mean(data, dim=[0, 2, 3])
std = torch.std(data, dim=[0, 2, 3])

print(f'Mean: {mean}\nStd: {std}')


Files already downloaded and verified
Mean: tensor([0.4914, 0.4822, 0.4465])
Std: tensor([0.2470, 0.2435, 0.2616])


In [14]:
test_transform = transforms.Compose([transforms.ToTensor(),
                                     transforms.Normalize(mean, std)
                                     ])
train_transform = transforms.Compose([transforms.RandomChoice(),
                                      ])